# Agent Spec: Demo Notebook

Welcome! This notebook walks through four examples with Agent Spec:

1. Structured Generation with an Llm Node to extract information from documents
2. Sequential operations with a Map Node to repeat the same operation on a list of inputs
3. ReAct-style Agent with tools (one using ServerTool, one using a RemoteTool that calls the REST Countries API)
4. ReAct-style Agent using MCP (Model Context Protocol) with a simple MCP SSE server

Target audience: Introduction level for students and developers new to Agent Spec

## 0) Installation & LLM Configuration

For installation, please follow the installation instructions from README.md, then restart the notebook kernel.

In [1]:
import warnings
warnings.filterwarnings('ignore')

Prepare the LLM model that is used in this tutorial.
In the cell below, we assume that you have a locally running Ollama model.
You can change the `model_id`, `url` and `default_generation_parameters`
You can find in Agent Spec documentation [how to use LLMs from other providers](https://oracle.github.io/agent-spec/howtoguides/howto_llm_from_different_providers.html).
You can also update the [generation config parameters](https://oracle.github.io/agent-spec/api/llmmodels.html#llmgenerationconfig)

In [5]:
from pyagentspec.llms import OpenAiCompatibleConfig, LlmGenerationConfig
import os
os.environ["OPENAI_API_KEY"] = "tgp_v1_vW09RC97sOgr4CxmYdfF9OF9LlY_ED73B8QFP4gzaA8"
llm_config = OpenAiCompatibleConfig(
    name="Together Model",
    model_id="openai/gpt-oss-20b",
    url="https://api.together.xyz/v1",
    default_generation_parameters=LlmGenerationConfig(
        max_tokens=512,
        temperature=0.7,
        top_p=0.95,
    )
)


To verify that the LLM config is correct and you have access to the model, we can execute it.
To execute Agent Spec configurations, a runtime is required. For this tutorial we will use the WayFlow runtime.

In [ ]:
from wayflowcore.agentspec import AgentSpecLoader

llm = AgentSpecLoader().load_component(llm_config)
completion = llm.generate("What is LauzHack?")
print(completion.message.content)

## 1) Structured Generation with an LLM Node

An `LlmNode` can be used to simply generate text completion from a prompt.
A more advanced usage is [Structured Generation](https://oracle.github.io/agent-spec/misc/glossary.html#structured-generation),
used too enforce the model to generate data of a desired format.
For example, in the LlmNode below, we configure it to not generate a reply but to produce 5 outputs:
`titles`, `topics`, `summaries`, `outlines`, `authors`.


In [ ]:
from pyagentspec.flows.nodes import LlmNode
from pyagentspec.property import StringProperty, ListProperty

article_analysis_node = LlmNode(
    name="structured_extract_node",
    llm_config=llm_config,
    prompt_template=(
        "Extract the relevant information from the Markdown article below:\n"
        "Article:\n{{article}}\n"
    ),
    outputs=[
        StringProperty(title="title", description="A concise, catchy title for the article"),
        StringProperty(title="topic", description="One or two words describing the main topic"),
        StringProperty(title="summary", description="A 2-3 sentence summary of the article"),
        StringProperty(title="outline", description="Bullet-style outline of the article's structure"),
        StringProperty(title="author", description="The author's name"),
    ],
)


### Load example Markdown articles
We will use three articles as dummy data to demo the Structured Generation process

In [ ]:
from pathlib import Path

article_paths = [
    Path("./articles/phone.md"),
    Path("./articles/laptop.md"),
    Path("./articles/headphones.md"),
]

articles = [p.read_text(encoding="utf-8") for p in article_paths]
print(f"Loaded {len(articles)} articles.")


In [ ]:
import json
from wayflowcore.agentspec import AgentSpecLoader
from wayflowcore.flow import Flow as ExecutableFlow

article_analysis = AgentSpecLoader().load_component(article_analysis_node)
conversation = ExecutableFlow.from_steps([article_analysis]).start_conversation({"article": articles[0]})
status = conversation.execute()

print(json.dumps(status.output_values, indent=2))


### 2) Sequential processing with a Map Node

The `MapNode` allows to repeat operations from a flow over multiple inputs. We wrap the above
`LlmNode` into a Flow, that we pass to the `MapNode` as its subflow. Then we place this `MapNode`
into a flow. See [our guide](https://oracle.github.io/agent-spec/howtoguides/howto_mapnode.html)
to go further.

In [ ]:
from pyagentspec.flows.flow import Flow
from pyagentspec.flows.edges import ControlFlowEdge, DataFlowEdge
from pyagentspec.flows.nodes import StartNode, EndNode

start_node = StartNode(name="start_node", inputs=[StringProperty(title="article", description="Markdown article content")])
end_node = EndNode(
    name="end_node",
    outputs=[
        StringProperty(title="title", description="A concise, catchy title for the article"),
        StringProperty(title="topic", description="One or two words describing the main topic"),
        StringProperty(title="summary", description="A 2-3 sentence summary of the article"),
        StringProperty(title="outline", description="Bullet-style outline of the article's structure"),
        StringProperty(title="author", description="The author's name"),
    ],
)

article_analysis_flow = Flow(
    name="mapnode_subflow",
    start_node=start_node,
    nodes=[start_node, article_analysis_node, end_node],
    control_flow_connections=[
        ControlFlowEdge(name="cfe1", from_node=start_node, to_node=article_analysis_node),
        ControlFlowEdge(name="cfe2", from_node=article_analysis_node, to_node=end_node),
    ],
    data_flow_connections=[
        DataFlowEdge(
            name="dfe1",
            source_node=start_node,
            source_output="article",
            destination_node=article_analysis_node,
            destination_input="article",
        ),
        DataFlowEdge(name="title", source_node=article_analysis_node, source_output="title", destination_node=end_node, destination_input="title"),
        DataFlowEdge(name="topic", source_node=article_analysis_node, source_output="topic", destination_node=end_node, destination_input="topic"),
        DataFlowEdge(name="summary", source_node=article_analysis_node, source_output="summary", destination_node=end_node, destination_input="summary"),
        DataFlowEdge(name="outline", source_node=article_analysis_node, source_output="outline", destination_node=end_node, destination_input="outline"),
        DataFlowEdge(name="author", source_node=article_analysis_node, source_output="author", destination_node=end_node, destination_input="author"),
    ],
)


### Serialize the Flow configuration

You can export the assistant configuration using the `AgentSpecSerializer`. Every `pyagentspec` component can be exported to a configuration. That allows to easily store or share them.

In [ ]:
from pyagentspec.serialization import AgentSpecSerializer

serialized_flow = AgentSpecSerializer().to_json(article_analysis_flow, indent=2)
print(serialized_flow)


### Define the MapNode and wrap it in a Flow

In [ ]:
from pyagentspec.flows.nodes import MapNode

# MapNode wrapping the sub-flow and collecting each output field
map_node = MapNode(
    name="map_node",
    subflow=article_analysis_flow,
)


In [ ]:
start_node = StartNode(name="start_node", inputs=[
    ListProperty(title="articles", description="List of articles", item_type=StringProperty(title="article"))
])
end_node = EndNode(
    name="end_node",
    outputs=[
        ListProperty(title="titles", description="A concise, catchy title for the article", item_type=StringProperty()),
        ListProperty(title="topics", description="One or two words describing the main topic", item_type=StringProperty()),
        ListProperty(title="summaries", description="A 2-3 sentence summary of the article", item_type=StringProperty()),
        ListProperty(title="outlines", description="Bullet-style outline of the article's structure", item_type=StringProperty()),
        ListProperty(title="authors", description="The author's name", item_type=StringProperty()),
    ],
)

sequential_analysis_flow = Flow(
    name="mapnode_subflow",
    start_node=start_node,
    nodes=[start_node, map_node, end_node],
    control_flow_connections=[
        ControlFlowEdge(name="cfe1", from_node=start_node, to_node=map_node),
        ControlFlowEdge(name="cfe2", from_node=map_node, to_node=end_node),
    ],
    data_flow_connections=[
        DataFlowEdge(name="dfe1", source_node=start_node, source_output="articles", destination_node=map_node, destination_input="iterated_article"),
        DataFlowEdge(name="title", source_node=map_node, source_output="collected_title", destination_node=end_node, destination_input="titles"),
        DataFlowEdge(name="topic", source_node=map_node, source_output="collected_topic", destination_node=end_node, destination_input="topics"),
        DataFlowEdge(name="summary", source_node=map_node, source_output="collected_summary", destination_node=end_node, destination_input="summaries"),
        DataFlowEdge(name="outline", source_node=map_node, source_output="collected_outline", destination_node=end_node, destination_input="outlines"),
        DataFlowEdge(name="author", source_node=map_node, source_output="collected_author", destination_node=end_node, destination_input="authors"),
    ],
)


### Serialize the Flow configuration
You can export the assistant configuration using the `AgentSpecSerializer`

In [ ]:
from pyagentspec.serialization import AgentSpecSerializer

serialized_flow = AgentSpecSerializer().to_json(sequential_analysis_flow, indent=2)
print(serialized_flow)


### Execute the Flow with WayFlow (non-conversational flow)

In [ ]:
from wayflowcore.agentspec import AgentSpecLoader

flow = AgentSpecLoader().load_json(serialized_flow)
conversation = flow.start_conversation({"articles": articles})
status = conversation.execute()

for article_idx, (title, topic, summary, outline, author) in enumerate(zip(*[
    status.output_values[k] for k in ["titles", "topics", "summaries", "outlines", "authors"]
])):
    print(f"\nArticle {article_idx+1}:")
    print("----------")
    print(f"Author: {author}")
    print(f"Title: {title}")
    print(f"Topic: {topic}")
    print(f"Summary: {summary}")
    print(f"Outline:\n{outline}")


## 2) ReAct Agent with Tools

We will now define ReAct-style conversational agents with tools
- First, with a ServerTool that can have any arbitrary implementation in Python
- Second, with a RemoteTool that configures a web request to an API

See the complete guide for [how to build ReAct Agents](https://oracle.github.io/agent-spec/howtoguides/howto_agents.html).

We then execute each agent interactively using the WayFlow loader.

### 2.a) Agent with ServerTool

In [ ]:
from pyagentspec.agent import Agent
from pyagentspec.tools import ServerTool
from pyagentspec.property import StringProperty

get_country_info_server_tool = ServerTool(
    name="get_country_info",
    description="Get basic country information (languages, capital, currencies, region, population).",
    inputs=[StringProperty(title="country", description="Name of a country")],
    outputs=[StringProperty(title="country_info")],
)

agent = Agent(
    name="Country Info Agent",
    llm_config=llm_config,
    tools=[get_country_info_server_tool],
    system_prompt="You are a helpful assistant that can think step-by-step and use tools.",
)


#### Interactive execution (ServerTool agent)
Type your message and press Enter. Type `quit` to stop.

In [ ]:
import requests
from wayflowcore.agentspec import AgentSpecLoader
from wayflowcore import MessageType

def get_country_info(country="Switzerland"):
    url = f"https://restcountries.com/v3.1/name/{country}"
    params = {
        "fields": "languages,capital,currencies,region,population,capitalInfo,name"
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    return str(response.json())

tool_registry = {
    "get_country_info": get_country_info,
}

executable_agent = AgentSpecLoader(tool_registry).load_component(agent)

conversation = executable_agent.start_conversation()
message_idx = -1
while True:
    conversation.execute()
    messages = conversation.get_messages()
    for message in messages[message_idx+1:]:
        if message.message_type == MessageType.TOOL_REQUEST:
            print(f"\n{message.message_type.value} >>> {message.tool_requests}")
        else:
            print(f"\n{message.message_type.value} >>> {message.content}")
    message_idx = len(messages)
    user_input = input(f"AGENT >>> {messages[-1].content}\n\n(write 'quit' or 'exit' to stop) USER >>> ")
    print(f"\nUSER >>> {user_input}")
    if user_input.strip().lower() in {"quit", "exit"}:
        print("\nBye!")
        break
    conversation.append_user_message(user_input)


### 2.b) Agent with RemoteTool

You can also read [the full guide for agents with RemoteTool](https://oracle.github.io/agent-spec/howtoguides/howto_agent_with_remote_tools.html)

In [ ]:
from pyagentspec.tools.remotetool import RemoteTool
from pyagentspec.property import StringProperty
from pyagentspec.agent import Agent


country_property = StringProperty(title="country", description="Country name", default="Switzerland")

remote_country_tool = RemoteTool(
    name="RESTCountries_Tool",
    description="Fetch country info using REST Countries. Provide country; return JSON string.",
    url="https://restcountries.com/v3.1/name/{{country}}?fields=languages,capital,currencies,region,population,capitalInfo,name",
    http_method="GET",
)

agent_remote = Agent(
    name="Country Info Agent",
    llm_config=llm_config,
    tools=[remote_country_tool],
    system_prompt="You are a helpful assistant that can think step-by-step and use tools.",
)


#### Interactive execution (RemoteTool agent)
Type your message and press Enter. For example, ask: "What is the capital of Switzerland?" Type `quit` to stop.

In [ ]:
from wayflowcore.agentspec import AgentSpecLoader
from wayflowcore import MessageType

executable_agent = AgentSpecLoader().load_component(agent_remote)

conversation = executable_agent.start_conversation()
message_idx = -1
while True:
    conversation.execute()
    messages = conversation.get_messages()
    for message in messages[message_idx+1:]:
        if message.message_type == MessageType.TOOL_REQUEST:
            print(f"\n{message.message_type.value} >>> {message.tool_requests}")
        else:
            print(f"\n{message.message_type.value} >>> {message.content}")
    message_idx = len(messages)
    user_input = input(f"AGENT >>> {messages[-1].content}\n\n(write 'quit' or 'exit' to stop) USER >>> ")
    print(f"\nUSER >>> {user_input}")
    if user_input.strip().lower() in {"quit", "exit"}:
        print("\nBye!")
        break
    conversation.append_user_message(user_input)


## 3) ReAct Agent using MCP (Model Context Protocol)

Goal: connect a ReAct-style agent to an MCP server that exposes tools, following the approach shown in the how-to guide. We'll use SSE transport to connect to a running MCP server.

### Run an MCP SSE Server

Start an MCP server that exposes tools and serves over SSE. You can adapt the example in `docs/pyagentspec/source/code_examples/howto_mcp.py` using `FastMCP`:

1. Create a small script that calls `create_server(host, port)` and then `server.run(transport=\"sse\")`.
2. Run it locally so it's reachable at `http://localhost:8080/sse` (or adjust the URL below if different).

In [ ]:
from pyagentspec.mcp import MCPTool, SSETransport
from pyagentspec.agent import Agent
from pyagentspec.serialization import AgentSpecSerializer

country_property = StringProperty(title="country", description="Country name", default="Switzerland")

# Point this to your MCP server's SSE endpoint
mcp_server_url = "http://localhost:8080/sse"

mcp_client = SSETransport(name="MCP Client", url=mcp_server_url)

get_country_info = MCPTool(
    name="get_country_info",
    description="Get basic country information (languages, capital, currencies, region, population).",
    inputs=[country_property],
    client_transport=mcp_client,
)

agent_mcp = Agent(
    name="Agent using MCP",
    llm_config=llm_config,
    system_prompt="You are a helpful assistant that can think step-by-step and use tools.",
    tools=[get_country_info],
)

serialized_agent_mcp = AgentSpecSerializer().to_json(agent_mcp, indent=2)
print(serialized_agent_mcp[:400] + ("..." if len(serialized_agent_mcp) > 400 else ""))


### Execute the MCP Agent with WayFlow

We load the AgentSpec JSON and start a conversation. Ask something that requires calling the MCP tools.

In [ ]:
from wayflowcore.agentspec import AgentSpecLoader
from wayflowcore import MessageType
from wayflowcore.mcp import enable_mcp_without_auth

enable_mcp_without_auth()
executable_agent = AgentSpecLoader().load_json(serialized_agent_mcp)

conversation = executable_agent.start_conversation()
message_idx = -1
while True:
    conversation.execute()
    messages = conversation.get_messages()
    for message in messages[message_idx+1:]:
        if message.message_type == MessageType.TOOL_REQUEST:
            print(f"\n{message.message_type.value} >>> {message.tool_requests}")
        else:
            print(f"\n{message.message_type.value} >>> {message.content}")
    message_idx = len(messages)
    user_input = input(f"AGENT >>> {messages[-1].content}\n\n(write 'quit' or 'exit' to stop) USER >>> ")
    print(f"\nUSER >>> {user_input}")
    if user_input.strip().lower() in {"quit", "exit"}:
        print("\nBye!")
        break
    conversation.append_user_message(user_input)


## Wrap-up

In this notebook, we went through:
- Structured Generation to control output formats for an LlmNode
- Sequential execution in a Flow using a MapNode
- ReAct agents with ServerTool and RemoteTool tools
- MCP integration shows how to connect an Agent to an MCP SSE server using `SSETransport` and `MCPTool`,
- Execution of Agent Spec components using the Wayflow Runtime

Visit our repository to learn more, build AI agents and contribute: https://github.com/oracle/agent-spec
